In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.48.136.57


In [ ]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 83.3 MB/s eta 0:00:00


In [ ]:
!streamlit run application.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇

⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.48.136.57:8501

⠏Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://violet-eels-sink.loca.lt
Extracting EDF parameters from /tmp/tmplc0b3kvq.edf...
EDF file detected
Setting channel info structure...
/content/application.py:58: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True)
/content/application.py:58: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True)
/content/application.py:58: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_path, prel

In [ ]:
import streamlit as st
import mne
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tempfile
import os

if 'is_trained' not in st.session_state:
    st.session_state.is_trained = False
if 'sleep_stage_classifier' not in st.session_state:
    st.session_state.sleep_stage_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
if 'sleep_disorder_classifier' not in st.session_state:
    st.session_state.sleep_disorder_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

disorder_labels = {0: "No Disorder", 1: "Insomnia", 2: "Sleep Apnea", 3: "Narcolepsy"}

def feature_extraction(eeg_data):
    features = eeg_data.reshape(1, -1)
    if features.shape[1] != 3000:
        if features.shape[1] < 3000:
            features = np.pad(features, ((0, 0), (0, 3000 - features.shape[1])), 'constant')
        else:
            features = features[:, :3000]
    return features

def train_models(X_train, y_train_sleep_stage, y_train_sleep_disorder):
    st.session_state.sleep_stage_classifier.fit(X_train, y_train_sleep_stage)
    st.session_state.sleep_disorder_classifier.fit(X_train, y_train_sleep_disorder)
    st.session_state.is_trained = True
    st.success("Models trained successfully.")

def predict_sleep_stage_and_disorder(eeg_data_sample):
    if not st.session_state.is_trained:
        st.error("Error: Models are not trained yet. Please train the models first.")
        return None, None

    features = feature_extraction(eeg_data_sample)
    predicted_sleep_stage = st.session_state.sleep_stage_classifier.predict(features)[0]
    predicted_sleep_disorder = st.session_state.sleep_disorder_classifier.predict(features)[0]
    return predicted_sleep_stage, disorder_labels[predicted_sleep_disorder]

st.title("Sleep Stage & Disorder Classification")

uploaded_psg = st.file_uploader("Upload PSG (EEG) file", type=["edf"])
uploaded_hypnogram = st.file_uploader("Upload Hypnogram file", type=["edf"])

if uploaded_psg and uploaded_hypnogram:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".edf") as tmp_psg:
        tmp_psg.write(uploaded_psg.read())
        psg_path = tmp_psg.name

    with tempfile.NamedTemporaryFile(delete=False, suffix=".edf") as tmp_hypnogram:
        tmp_hypnogram.write(uploaded_hypnogram.read())
        hypnogram_path = tmp_hypnogram.name

    raw = mne.io.read_raw_edf(psg_path, preload=True)
    raw.pick_channels(['EEG Fpz-Cz'])
    raw.resample(100)
    eeg_data = raw.get_data()[0]

    annotations = mne.read_annotations(hypnogram_path)
    sleep_stages = []

    for desc, onset, duration in zip(annotations.description, annotations.onset, annotations.duration):
        if 'Sleep stage' in desc:
            try:
                stage = int(desc.replace('Sleep stage ', ''))
                num_epochs = int(duration / 30)
                sleep_stages.extend([stage] * num_epochs)
            except ValueError:
                continue

    X = np.random.randn(1000, 3000)
    y_sleep_stage = np.random.randint(1, 6, size=1000)
    y_sleep_disorder = np.random.randint(0, 4, size=1000)

    if st.button("Train Models"):
        train_models(X, y_sleep_stage, y_sleep_disorder)

    sample_index = st.number_input("Enter the index of the EEG sample (0 to 999)", min_value=0, max_value=999, value=0)
    if st.button("Predict Sleep Stage & Disorder"):
        eeg_data_sample = eeg_data[sample_index]
        predicted_sleep_stage, predicted_sleep_disorder = predict_sleep_stage_and_disorder(eeg_data_sample)
        if predicted_sleep_stage is not None:
            st.write(f"**Predicted Sleep Stage:** {predicted_sleep_stage}")
            st.write(f"**Predicted Sleep Disorder:** {predicted_sleep_disorder}")

    os.remove(psg_path)
    os.remove(hypnogram_path)